In [1]:
!which python3
import os
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from dataset_spectrogram import EEGDataset
from torch.utils.data import random_split
import neptune.new as neptune
from torchinfo import summary
from dataset_spectrogram import load_dataset
import random
import torch.utils.data as data
from datetime import datetime

/opt/anaconda-2021.11/bin/python3
Spectrogram dataset version 19 log


In [2]:
# load in the dataset


raw_data_dir = '../data'

trainingNights = 70
testNights = 8

print("\nTest set\n")
test_set = load_dataset(range(testNights), raw_data_dir, normalized = False)

print("\n\nTraining set\n")
training_set = load_dataset([x+testNights for x in range(trainingNights)], raw_data_dir, normalized = False)





Test set

../data/study_1A_mat_simple/S_01/night_1/spectrogram_bad_segments_unnormalized.npy
../data/study_1A_mat_simple/S_01/night_1/spectrogram_good_segments_unnormalized.npy
Memory usage: 0.037553 MB

Lengths:

Good data length: 7368
Bad data length: 7368
Caluculated length: 14735
../data/study_1A_mat_simple/S_01/night_2/spectrogram_bad_segments_unnormalized.npy
../data/study_1A_mat_simple/S_01/night_2/spectrogram_good_segments_unnormalized.npy
Memory usage: 0.043823 MB

Lengths:

Good data length: 4292
Bad data length: 4292
Caluculated length: 8583
../data/study_1A_mat_simple/S_01/night_3/spectrogram_bad_segments_unnormalized.npy
../data/study_1A_mat_simple/S_01/night_3/spectrogram_good_segments_unnormalized.npy
Memory usage: 0.047606 MB

Lengths:

Good data length: 6110
Bad data length: 6110
Caluculated length: 12219
../data/study_1A_mat_simple/S_01/night_4/spectrogram_bad_segments_unnormalized.npy
../data/study_1A_mat_simple/S_01/night_4/spectrogram_good_segments_unnormalized.np

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' #Check for cuda 
print(f'Using {device} device')



class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        #self.flatten = nn.Flatten()
        self.conv_stack = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.Conv2d(128,64,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Flatten(),
            nn.Linear(9920, 512),
            nn.ReLU(),
            nn.Linear(512,1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        #x = self.flatten(x)
        #x = torch.log(x)
        logits = self.conv_stack(x)
        return logits

Using cuda device


In [4]:
learning_rate = 1e-6
batch_size = 64
weight_decay = 0


model = NeuralNetwork()
model.to(device)

loss = nn.BCEWithLogitsLoss() 

#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay = weight_decay)

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    
    for batch, (X, y) in enumerate(dataloader):
        X = X[:,None,:,:].to(device)
        y = y.to(device)
        # Compute prediction and loss
        pred = model(X) 
        pred = pred.reshape(-1)
        pred = pred.to(device)
        yFloat = y.type(torch.FloatTensor).to(device)
        
        loss = loss_fn(pred, yFloat)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Neptune logging
        run["training/batch/loss"].log(loss)
        

        if batch % 1000 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        
        if batch % 4000 == 0:
            print(f"Predicted values: \n{pred}")
            print(f"Actual values: \n{yFloat}")
            print(f"Difference: \n{(yFloat-pred)}")


def test_loop(dataloader_test, model, loss_fn, test_set = True):
    size = len(dataloader_test.dataset)
    num_batches = len(dataloader_test)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader_test:
            X = X[:,None,:,:].to(device)
            y = y.to(device)
            pred = model(X).reshape(-1).to(device) # Reshape to 1 dimension if using binary classification, otherwise keep dimensions from model output
            test_loss += loss_fn(pred, y.type(torch.FloatTensor).to(device)).item()
            correct += (pred.round() == y).type(torch.float).sum().item()
            

    test_loss /= num_batches
    correct /= size
    
    
    
    if test_set:
        print(f"Test set Error: \n Test Set Accuracy: {(100*correct):>0.5f}%, Avg Test Set loss: {test_loss:>8f} \n")
        
        # Neptune logging
        run["testing/batch/test_loss"].log(test_loss)
        run["testing/batch/test_Acc"].log(100*correct)
    
    else:
        print(f"Training Set Error: \n Training Set Accuracy: {(100*correct):>0.5f}%, Avg Training Set loss: {test_loss:>8f} \n")
        
        # Neptune logging
        run["testing/batch/training_loss"].log(test_loss)
        run["testing/batch/training_Acc"].log(100*correct)
    


In [6]:
train_dataloader = DataLoader(training_set, batch_size=64, drop_last = True) # Drop_last, to avoid incomplete batches, which won't work with weighted loss
test_dataloader = DataLoader(test_set, batch_size=64, drop_last = True) # Drop_last, to avoid incomplete batches, which won't work with weighted loss


In [7]:

run = neptune.init(
    project="NTLAB/artifactDetect-ear", 
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxYTA4NzcxMy1lYmQ2LTQ3NTctYjRhNC02Mzk1NjdjMWM0NmYifQ==", # your credentials
    source_files=["train_model_spectrogram.ipynb", "dataset_spectrogram.py"]
)  # Credentials


run['config/dataset/size'] = trainingNights 
run['config/model'] = type(model).__name__
run['config/modelSummary'] = str(model)
run['config/optimizer'] = type(optimizer).__name__
run['config/batch_size'] = batch_size
run['config/test_night'] = testNights
run['config/learning_rate'] = learning_rate
run['config/weight_decay'] = weight_decay


https://app.neptune.ai/NTLAB/artifactDetect-ear/e/AR-64
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [ ]:
epochs = 200
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    
    train_loop(train_dataloader, model, loss, optimizer)       
    test_loop(test_dataloader, model, loss)
    
    if (t % 3) == 0:# Only run training accuracy test every third time
        test_loop(train_dataloader, model, loss, test_set = False)
    if (t % 20) == 0:
        now = datetime.now()

        print(f"../trained_models/model_{now.strftime('%m_%d_%Y_%H_%M_%S')}")

        model_scripted = torch.jit.script(model) # Export to TorchScript
        model_scripted.save(f"../trained_models/model_{now.strftime('%m_%d_%Y_%H_%M_%S')}") # Save

        
        
now = datetime.now()

print(f"../trained_models/model_{now.strftime('%m_%d_%Y_%H_%M_%S')}")

model_scripted = torch.jit.script(model) # Export to TorchScript
model_scripted.save(f"../trained_models/model_{now.strftime('%m_%d_%Y_%H_%M_%S')}") # Save

print("Done!")

Epoch 1
-------------------------------
loss: 0.699127  [    0/1690404]
Predicted values: 
tensor([7.1475e-01, 5.2176e-01, 6.8673e-01, 5.4532e-01, 2.0719e-01, 6.2468e-01,
        8.7544e-01, 5.4347e-01, 6.2511e-01, 0.0000e+00, 9.9754e-01, 5.2008e-01,
        1.2348e-02, 5.1173e-01, 5.3073e-01, 5.1330e-01, 9.9996e-01, 5.0021e-01,
        7.1507e-01, 5.1180e-01, 5.7006e-01, 5.0842e-01, 9.3790e-01, 9.9991e-01,
        7.1765e-01, 5.1102e-01, 2.3405e-04, 5.0983e-01, 1.0000e+00, 5.1677e-01,
        1.0000e+00, 5.0943e-01, 9.5166e-01, 4.6639e-01, 7.1583e-01, 5.3213e-01,
        6.5269e-01, 5.0824e-01, 7.6914e-01, 5.1232e-01, 6.6873e-01, 5.3938e-01,
        8.2692e-01, 5.1541e-01, 5.9634e-01, 5.2131e-01, 7.5836e-01, 4.8160e-01,
        6.9791e-01, 5.5286e-01, 9.8916e-01, 5.3644e-01, 8.4159e-01, 5.4301e-01,
        6.2014e-01, 5.0567e-01, 5.8503e-01, 5.1193e-01, 5.2303e-01, 5.1851e-01,
        7.2942e-01, 5.5139e-01, 9.9941e-01, 5.2246e-01], device='cuda:0',
       grad_fn=<ReshapeAliasBackwar

In [ ]:
# Test the model, and inspect the errors
from datetime import datetime
now = datetime.now()

print(f"../trained_models/model_{now.strftime('%m_%d_%Y_%H_%M_%S')}")

model_scripted = torch.jit.script(model) # Export to TorchScript
model_scripted.save(f"../trained_models/model_{now.strftime('%m_%d_%Y_%H_%M_%S')}") # Save



In [ ]:
run.stop() # Stop the neptune logging run

In [ ]:
for batch, (X, y) in enumerate(train_dataloader):
    if sum(sum(sum(np.isnan(X)))):
        print(batch)
        print("X:   NAN FOUND!!!\nNAN FOUND!!!\nNAN FOUND!!!\nNAN FOUND!!!\nNAN FOUND!!!\nNAN FOUND!!!\nNAN FOUND!!!\n")
        print(sum(sum(sum(np.isnan(X)))))
        
    if batch % 1000 == 100:
        print(batch)
    

In [ ]:
for index, i in enumerate(training_set):
    if sum(sum(np.isnan(i[0]))):
        #print(index)
        #print(i)
        print(f"X:   NAN FOUND!!! At index: {index}. Amount of nans: {sum(sum(np.isnan(i[0])))}")
        #print(sum(sum(np.isnan(i[0]))))
        
    if index % 50000 == 0:
        print(index)
    